In [1]:
# Import packages
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import datetime
from datetime import datetime, timedelta
from shapely.geometry import Point
from shapely.geometry import shape
from statsmodels.tsa.arima.model import ARIMA
from keras.models import Sequential
from keras.layers import Dense
import xgboost as xgb
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, MinMaxScaler
import libpysal as ps
from spreg import GM_Lag
from sklearn.linear_model import LassoCV
import statsmodels.api as sm
import os
from zipfile import ZipFile
import random
import networkx as nx
import warnings

In [2]:
# Open the zip file in read-only mode.

DC_boundary = gpd.read_file('Data/Shapefile/Washington_DC_Boundary.geojson')

def update_datetime(row):
    if int(row[11:13]) >= 24:
        return datetime.strptime(row[:11] + str(int(row[11:13])%24) + row[13:], '%Y-%m-%d %H:%M:%S') + timedelta(days = int(row[11:13])//24) # Increment the day
    else:
        return datetime.strptime(row, '%Y-%m-%d %H:%M:%S')

directory = "Data/GTFS/"
file_paths = []
# crawling through directory and subdirectories
for root, directories, files in os.walk(directory):
    for filename in files:
        # join the two strings in order to form the full filepath.
        filepath = os.path.join(root, filename)
        file_paths.append(filepath)

date_list = []
for i in range (len(file_paths)):
    # Extract the date from the file name
    date_str = os.path.splitext(os.path.basename(file_paths[i]))[0].split('_')[1]
    # Convert the date string to datetime object
    date_obj = datetime.strptime(date_str, '%Y%m%d')
    # Format the datetime object as 'YYYY-MM-DD'
    date_list.append(date_obj.strftime('%Y-%m-%d') + ' ')

transit_stops_info = []
for i in range (154, 231):
    archive = ZipFile(file_paths[i], "r")
    trips = pd.read_csv(archive.open('trips.txt'))
    stops = pd.read_csv(archive.open('stops.txt'))
    ## add geometry for stops
    stops['geometry'] = stops[['stop_lon', 'stop_lat']].values.tolist()
    stops['geometry'] = stops['geometry'].apply(Point)
    # select stops within DC
    DC_stop_index = []
    for j in range (len(stops)):
        if stops.geometry[j].within(DC_boundary.geometry[0]):
            DC_stop_index.append(j)
    stops = stops.loc[DC_stop_index]
    stops_time = pd.read_csv(archive.open('stop_times.txt'))
    ## merge
    transit_stops_time = stops_time.merge(stops, how = 'left', on = "stop_id")
    transit_stops_time = transit_stops_time.merge(trips, how = 'left', on = "trip_id")
    # select trip id within DC
    transit_stops_time = transit_stops_time.iloc[np.isin(transit_stops_time['stop_id'], stops.stop_id.unique())]
    # delete unnecessary columns
    transit_stops_time = transit_stops_time.drop(columns = {"trip_id", "stop_sequence", "pickup_type", "drop_off_type", "shape_dist_traveled", "route_id", "service_id", "stop_code", "stop_desc", "stop_name", "zone_id", "block_id", "scheduled_trip_id", "trip_headsign", "direction_id", "shape_id"}, axis = 1)
    # add date information for the dataframe
    transit_stops_time.arrival_time = date_list[i] + transit_stops_time.arrival_time
    transit_stops_time.departure_time = date_list[i] + transit_stops_time.departure_time
    transit_stops_info.append(transit_stops_time)
    print(i)

transit_stops_info = pd.concat(transit_stops_info, ignore_index = True)
transit_stops_info["arrival_time"] = transit_stops_info["arrival_time"].apply(update_datetime)
transit_stops_info["arrival_time"] = pd.to_datetime(transit_stops_info["arrival_time"])
transit_stops_info["departure_time"] = transit_stops_info["departure_time"].apply(update_datetime)
transit_stops_info["departure_time"] = pd.to_datetime(transit_stops_info["departure_time"])
print(len(transit_stops_info))
transit_stops_info.head()

C:\Users\Kaifa Lu\AppData\Local\Temp\ipykernel_13148\2280318006.py:32: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv(archive.open('trips.txt'))
C:\Users\Kaifa Lu\AppData\Local\Temp\ipykernel_13148\2280318006.py:43: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  stops_time = pd.read_csv(archive.open('stop_times.txt'))


154


C:\Users\Kaifa Lu\AppData\Local\Temp\ipykernel_13148\2280318006.py:32: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv(archive.open('trips.txt'))
C:\Users\Kaifa Lu\AppData\Local\Temp\ipykernel_13148\2280318006.py:43: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  stops_time = pd.read_csv(archive.open('stop_times.txt'))


155


C:\Users\Kaifa Lu\AppData\Local\Temp\ipykernel_13148\2280318006.py:32: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv(archive.open('trips.txt'))
C:\Users\Kaifa Lu\AppData\Local\Temp\ipykernel_13148\2280318006.py:43: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  stops_time = pd.read_csv(archive.open('stop_times.txt'))


...

C:\Users\Kaifa Lu\AppData\Local\Temp\ipykernel_13148\2280318006.py:32: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv(archive.open('trips.txt'))
C:\Users\Kaifa Lu\AppData\Local\Temp\ipykernel_13148\2280318006.py:43: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  stops_time = pd.read_csv(archive.open('stop_times.txt'))


204


C:\Users\Kaifa Lu\AppData\Local\Temp\ipykernel_13148\2280318006.py:32: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv(archive.open('trips.txt'))
C:\Users\Kaifa Lu\AppData\Local\Temp\ipykernel_13148\2280318006.py:43: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  stops_time = pd.read_csv(archive.open('stop_times.txt'))


205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
139423475


,arrival_time,departure_time,stop_id,stop_lat,stop_lon,geometry
0,2019-07-23 07:24:00,2019-07-23 07:24:00,5660,38.947849,-77.079299,POINT (-77.079299 38.947849)
1,2019-07-23 07:27:00,2019-07-23 07:27:00,4941,38.944519,-77.063716,POINT (-77.063716 38.944519)
2,2019-07-23 07:29:00,2019-07-23 07:29:00,4912,38.935852,-77.058624,POINT (-77.058624 38.935852)
3,2019-07-23 07:31:00,2019-07-23 07:31:00,4905,38.925000,-77.052350,POINT (-77.05235 38.925)
4,2019-07-23 07:33:00,2019-07-23 07:33:00,4956,38.911129,-77.044428,POINT (-77.044428 38.911129)


In [3]:
transit_stops_info.to_csv("Data/transit_stops_info_part3.csv", index = False)